In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import string
import random
import numpy as np
import pandas as pd
import tensorflow as tf

/home/florian/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Creating fake Pokemon names with RNNs

An interesting property of recurrent neural networks is their ability to generate sequences. We see this when using an RNN for image captioning, language translation, sequence tagging, or sequence forecasting. To help understand this method we'll use a character level RNN model the sequences of letters that make up a Pokemon's name and type. A technique popularized by Andre Karpathy in his blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/). 

## Load the provided Pokemon csv file with pandas and view it's structure

In [2]:
# load in some text to use
poke_df = pd.read_csv('pokemon/Pokemon.csv')
poke_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


### Make a list of strings of just pokemon names and their types

We append an 'End of Sentence' token to each pokemon name. When we end up generating the pokemon names the network will output that EOS token when it's done making up a name. 

In [3]:
def strip_non_ascii(some_string):
    return ''.join([c for c in some_string if c in string.printable])

pokemons = ["{} {}<EOS>".format(df_row[1]['Name'], df_row[1]['Type 1']) for df_row in poke_df.iterrows()]
pokemons = [strip_non_ascii(pokemon) for pokemon in pokemons]
print(pokemons[0])

Bulbasaur Grass<EOS>


### Function for creating training data

For this task we will go over each character in a string and train the RNN to predict the following character. So set up our input and targets for this type of training we simply take all but the last character for the input and all but the first character for the output. Then we'll pair up those sequences for our (input, target) pairs. 

This will be effective because once we have the model trained we can generate fake names by starting with a randomly generated letter and feeding that to the network. The network will then predict a reasonable next character, which is then fed back into the network to generate the next letter. This goes on until we reach the EOS token.

In [4]:
def list_iter(pokemon_names):
    inp = pokemon_names[:-1] # all but last
    targ = pokemon_names[1:] # all but first
    
    return inp, targ

Neural networks can not work on characters directly. Instead we need some numerical representation of those characters. So we'll make a list of every printable character and use that to create dictionaries to convert a letter to an integer and another to convert back to the character.

In [5]:
# need to get all of the possible characters that the source uses
chars = string.printable

data_size, vocab_size = len(pokemons), len(chars)
print('Text is', data_size, 'pokemon long and there are', vocab_size, 'unique characters.')

Text is 800 pokemon long and there are 100 unique characters.


In [6]:
# create dictionaries to convert from characters to index and from index back to characters
char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for i, ch in enumerate(chars)}

# Building an RNN in tensorflow

In this notebook we'll work in Tensorflow directly. I would recommend getting familiar with how neural networks work by using our previous examples and then once you feel comfortable with Keras and all of the high level concepts move into Tensorflow. 

Tensorflow does give us a few helper functions to facilitate the construction of neural networks, but mostly we will be building lots of things from scratch. The one thing that we definitely don't want to do is calculate the backward pass for our training steps, luckily this is something that Tensorflow will do for us. 

In this example we will create a GRU recurrent neural network to use in our character level RNN. The steps for creating this network from scratch will be:

* Initialize all of our weight matrices. Setup their sizes and fill with random numbers
* Define the calculations that our network must carry out

A GRU cell is basically a change in the way that the hidden state is calculated for a recurrent neural network. So to begin we'll start with a vanilla recurrent neural network and show how we can create one using the two steps above. 

### Vanilla RNN

The calculations for a recurrent neural network look like the following:

![rnn](images/rnn.png)

In order to create that we need to set up three matrices and two bias vectors. The specify the calculations in exactly the same way. 

```python
Uh = tf.get_variable("Uh", [input_size, hidden_size], initializer=tf.random_normal_initializer(stddev=0.1))
Wh = tf.get_variable("Wh", [hidden_size, hidden_size], initializer=tf.random_normal_initializer(stddev=0.1))
Vy = tf.get_variable("Vy", [hidden_size, vocab_size], initializer=tf.random_normal_initializer(stddev=0.1))
bh  = tf.get_variable("bh", [hidden_size], initializer=tf.random_normal_initializer(stddev=0.1))
by  = tf.get_variable("by", [output_size], initializer=tf.random_normal_initializer(stddev=0.1))

hs_t = tf.tanh(tf.matmul(xs_t, Uh) + tf.matmul(hs_t, Wh) + bh)
ys_t = tf.nn.softmax(tf.matmul(hs_t, Vy) + by)
```

Simple enough. Input_size and output_size will change depending on the properties of our data. Hidden_size is a hyperparameter that we can set to anything that we wish.

In [7]:
# define some hyperparameters for our network
embed_size = 64
hidden_size = 128
epochs = 50

## Exercise: Set up the RNN and Train it to generate pokemon names

Whenever we run a RNN for a step we get two outputs. The output of the RNN and it's current hidden state. On the next step, we have to pass both the data at that step and the hidden state from the previous step. Therefore, we need to set up a placeholder to recieve the hiddent states at each step. This will be like having two inputs to our network, both the input and hidden state need to be fed with the feed_dict.

In [8]:
# set up the place holders for our computational graph
inputs = tf.placeholder(shape=[None, 1], dtype=tf.int32, name='input')
targets = tf.placeholder(shape=[None, vocab_size], dtype=tf.float32, name='targets')
init_state = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32, name='state')

# create an initializer to init our weight matricies
init = tf.random_normal_initializer(stddev=0.1)

### Embeddings

Passing integers to a neural network is not very useful. All of our operations are typically matrix operations of some kind. Performing those operations with an integer does not give us a lot of flexibility in the kinds of representations that the network can learn. Instead we will put something called and `embedding layer` at the start of our network. For this purpose this will be of size `[vocab_size, embedding size]`, where vocab is the number of possible characters and embedding size is a hyper parameter. This matrix will be initialized with random numbers. The embedding layer will be a lookup table. The index representing the character will correspond to the index of some vector within that matrix. That vector will then be used to represent our character to the network. This way we perform a vector-matrix operation when processing a character. The values in the embedding matrix will update through backpropagation just like the rest of the network. They will end up becoming meaningful vector representations of the characters. 

In [9]:
# set up the embedding layer
with tf.device('/cpu:0'), tf.name_scope("Embedding"):
    embedding = tf.get_variable("embedding", [vocab_size, embed_size], initializer=init)
    inputs_embedding = tf.nn.embedding_lookup(embedding, inputs)
    inputs_embedding = tf.reshape(inputs_embedding, (1, embed_size))

# set up our recurrent neural network and define the functions
with tf.variable_scope("RNN") as scope:
    # hidden state at time t 
    hs_t = init_state
            
    Uh = tf.get_variable("Uh", [embed_size, hidden_size], initializer=tf.random_normal_initializer(stddev=0.1))
    Wh = tf.get_variable("Wh", [hidden_size, hidden_size], initializer=tf.random_normal_initializer(stddev=0.1))
    Vy = tf.get_variable("Vy", [hidden_size, vocab_size], initializer=tf.random_normal_initializer(stddev=0.1))
    bh  = tf.get_variable("bh", [hidden_size], initializer=tf.random_normal_initializer(stddev=0.1))
    by  = tf.get_variable("by", [vocab_size], initializer=tf.random_normal_initializer(stddev=0.1))

    hs_t = tf.tanh(tf.matmul(inputs_embedding, Uh) + tf.matmul(hs_t, Wh) + bh)
    ys_t = tf.matmul(hs_t, Vy) + by


In [10]:
# need to keep track of our hidden states
h_0 = hs_t
# apply the softmax output to the last output of our list
output_softmax = tf.nn.softmax(ys_t)

# get all of the output characters together
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=ys_t))

# optimization algorithm
optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
grads = optimizer.compute_gradients(loss)

# clip the gradients
grad_clipping = tf.constant(5.0, name='grad_clipping')
clipped_grads = []
for grad, var in grads:
    clipped_grad = tf.clip_by_value(grad, -grad_clipping, grad_clipping)
    clipped_grads.append((clipped_grad, var))
    
# update the weights with gradient descent
updates = optimizer.apply_gradients(clipped_grads)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [11]:
# now that all the functions are set up we can run this thing

# function to one hot encode the characters
def one_hot(v):
    return np.eye(vocab_size)[v]

# Session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# Initial values
n, p = 0, 0
#hprev_val = np.zeros([1, hidden_size])

for _ in range(epochs):
    for pokemon in pokemons:
        # Initialize the hidden state to 0 at the beginning of each sequence
        h_t = np.zeros([1, hidden_size])

        # Prepare inputs
        input_vals, target_vals = list_iter(pokemon)

        input_vals = [char2idx[c] for c in input_vals]
        target_vals = [char2idx[c] for c in target_vals]

        target_vals = one_hot(target_vals)
   
        losses = []
        for c in range(len(input_vals)):
            # run the tensorflow session
            h_t, loss_val, _ = sess.run([h_0, loss, updates],
                                        feed_dict={inputs: np.asarray(input_vals[c]).reshape(1,1),
                                                   targets: target_vals[c].reshape(1,100),
                                                   init_state: h_t})
        losses.append(loss_val)
        if n % 1000 == 0:
            # Progress
            print('iter: %d, p: %d, loss: %f' % (n, p, np.mean(losses)))

            # Do sampling
            sample_length = 50
            prime_str_idx = np.random.randint(len(string.ascii_uppercase))
            prime_str = string.ascii_uppercase[prime_str_idx]

            idxs = []
            sample_prev_state_val = np.copy(h_t)
            sample_input_vals = np.asarray([char2idx[prime_str]]).reshape(1,1)

            for t in range(sample_length):
                sample_output_softmax_val, sample_prev_state_val = \
                    sess.run([output_softmax, h_0],
                             feed_dict={inputs: sample_input_vals, init_state: sample_prev_state_val})

                predicted_idx = (np.argmax(sample_output_softmax_val))

                idxs.append(predicted_idx)
                sample_input_vals = np.asarray([predicted_idx]).reshape(1,1)

            txt = prime_str + ''.join(idx2char[ix] for ix in idxs)
            print('----\n %s \n----\n' % (txt.split('<EOS>')[0],))

        n += 1

iter: 0, p: 0, loss: 4.160430
----
 O;ErarDEE;rpBrEzr;	:ErjEE.r Br6zj;E;Fr;B>.r;	J;rjBr 
----

iter: 1000, p: 0, loss: 0.000246
----
 Dadolos Electric 
----

iter: 2000, p: 0, loss: 0.000025
----
 Holie Ice 
----

iter: 3000, p: 0, loss: 0.000108
----
 Carditord Water 
----

iter: 4000, p: 0, loss: 0.000014
----
 Flant Rock 
----

iter: 5000, p: 0, loss: 0.000010
----
 Quctrote Fire 
----

iter: 6000, p: 0, loss: 0.000005
----
 Zunt Ice 
----

iter: 7000, p: 0, loss: 0.000438
----
 Non Grass 
----

iter: 8000, p: 0, loss: 0.000001
----
 Fle Size Ghost 
----

iter: 9000, p: 0, loss: 0.000003
----
 Gradoss Electric 
----

iter: 10000, p: 0, loss: 0.000000
----
 Fine Ghost 
----

iter: 11000, p: 0, loss: 0.000013
----
 Kino Normal 
----

iter: 12000, p: 0, loss: 0.000000
----
 Vicoom Dark 
----

iter: 13000, p: 0, loss: 0.000002
----
 OSwin Fire 
----

iter: 14000, p: 0, loss: 0.000002
----
 Qlepich Ice 
----

iter: 15000, p: 0, loss: 0.000008
----
 Hoad Formal 
----

iter: 16000, p: 0, l